In [1]:
IS_LINUX = False
from sys import platform
if platform == "linux" or platform == "linux2":
  IS_LINUX = True
if IS_LINUX:
    !pip install tensorflow-gpu
    !pip install jsonpickle

     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 3.8MB 37.6MB/s 
     |████████████████████████████████| 450kB 41.8MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exist

In [2]:
if IS_LINUX:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
if IS_LINUX:
    %cd /content/drive/My\ Drive/MastersThesis
    !ls
    !pwd
    import sys
    sys.path.append('./')

/content/drive/My Drive/MastersThesis
 Autoencoderstart.ipynb   model_combined.png
 callbacks.py		  models.py
 config.py		  named_constants.py
 constants.py		  output_dir
 datasets.py		  __pycache__
 experiments.py		  README.md
 for_fun.py		  training_data.py
 for_training.ipynb	  vizualizer.py
 global_functions.py	 'Vysledky Fashion mnist 22.10.2019 basic model.xlsx'
/content/drive/My Drive/MastersThesis


In [10]:
try:
    import google.colab
    IS_COLAB = True
except:
    IS_COLAB = False
IS_COLAB

True

In [0]:
import global_functions
global_functions.on_start()

In [0]:
import tensorflow as tf
print(tf.__version__)
#2.0.0

In [0]:
import experiments, datasets, training_data
import models
import constants
import vizualizer

from global_functions import on_start
on_start()

for repeat in range(1):
    for rate in [0.1, 0.25, 0.5, 1]:
        exp = experiments.ExperimentAutoencoder(datasets.MnistDataset, models.BasicModelProvider(),
                                                training_data.BasicTrainingDataGeneratorAutoencoder()
                                                )
        exp.train(training_data.BasicTrainParametersAutoencoder(100, 128, 20, True, rate, 1e-5,
                                                                constants.ExperimentsPaths.Mnist.AUTOENCODER,
                                                                True))

In [0]:
import experiments, datasets, training_data
import models
import constants
import vizualizer

from global_functions import on_start
on_start()
# 2
for repeat in range(1):
    for rate in [0.1, 0.25, 0.5, 1]:
        data_provider = training_data.BasicTrainingDataGeneratorClassifier
        exp = experiments.ExperimentClassifier(datasets.MnistDataset, models.BasicModelProvider(),
                                               training_data.BasicTrainingDataGeneratorClassifier())
        exp.train(training_data.BasicTrainParametersClassifier(100, 128, 20, True, rate, 1e-6, True,
                                                               constants.ExperimentsPaths.Mnist.CLASSIFIER, True)
                  )

In [0]:
import vizualizer, experiments, constants, datasets

results = experiments.ExperimentBase.load_experiment_results(constants.ExperimentsPaths.Mnist.CLASSIFIER, False)
vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)


In [0]:
def make_batches(to_batch, batch_size = 128):
    return [to_batch[i * batch_size:(i + 1) * batch_size] for i in range((len(to_batch) + batch_size - 1) // batch_size )]

batch_size = 128
to_batch = tf.data.Dataset.from_tensor_slices((train_images, keras.utils.to_categorical(train_labels, 10) ))
train_batched = to_batch.batch(batch_size)

In [0]:
epochs = 10
for epoch in range(epochs):
    print("Epoch: {}", epoch)
    for step, (images, labels) in enumerate(train_batched):
        set_autoencoder_trainable(True)
        with tf.GradientTape() as tape:
          prediction = autoencoder(images)
          loss_autoencoder = autoencoder.loss(images, prediction)
          gradients = tape.gradient(loss_autoencoder, autoencoder.trainable_variables)
          autoencoder.optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))  
        set_autoencoder_trainable(False)
        with tf.GradientTape() as tape:
          predict_class = classifier(images)
          loss_classifier = classifier.loss(labels, predict_class)  
#           classiefier_trainable = classifier_head.trainable_variables
          classiefier_trainable = classifier.trainable_variables
          gradients = tape.gradient(loss_classifier, classiefier_trainable)
          classifier.optimizer.apply_gradients(zip(gradients  ,classiefier_trainable) ) 
          if step % 10 == 0:
              print("Step: {}, loss autoencoder: {}, loss classifier: {}".format(step, loss_autoencoder.numpy().mean(), loss_classifier.numpy().mean()))


In [0]:
for epoch in range(5):
  print("Classifier")
  historyClassifier = classifier.fit(x = train_images, y = train_labels_one_hot_encoding, batch_size=128, epochs=1, validation_data= (test_images, test_labels_one_hot_encoding))

In [0]:
predictions_auto_encoder, predictions_classifier = auto_classifier.predict(test_images)
predictions_classifier = np.argmax(np.round(predictions_classifier),axis=1)
correct = np.where(predictions_classifier==test_labels)[0]
print( "Found {} correct labels".format(len(correct)))

In [0]:

test_eval = classifier.evaluate(test_images, test_labels_one_hot_encoding, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])
predicted_classes = classifier.predict(test_images)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes = predicted_classes.astype('uint8')
correct = [predicted_classes[i] == value for i, value in enumerate(test_labels)]
print("Correct: {}".format( correct.count(True)))




In [0]:
predictions = autoencoder.predict(test_images)
# rand_start = np.random.randint(0, len(predictions) - 10)
# for i in range(rand_start, rand_start + 10):
#   plotImage(predictions[i].reshape(28, 28), "Predictions")
#   plotImage(test_images[i].reshape(28, 28), "Original")


num_images = 5
#np.random.seed(42)
random_test_images = np.random.randint(test_images.shape[0], size=num_images)

decoded_imgs = predictions


plt.figure(figsize=(28, 16))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(2, num_images, i + 1)
    plt.title("Original")
    plt.imshow(process_for_plotting(test_images[image_idx]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # # plot encoded image
    # ax = plt.subplot(3, num_images, num_images + i + 1)
    # plt.imshow(encoded_imgs[image_idx].reshape(8, 4))
    # plt.gray()
    # ax.get_xaxis().set_visible(False)
    # ax.get_yaxis().set_visible(False)

    # plot reconstructed image
    ax = plt.subplot(2, num_images, num_images + i + 1)
    plt.imshow(process_for_plotting(decoded_imgs[image_idx])) #reshape(28, 28)
    plt.title("Predicted")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:

from experiments import *
import jsonpickle
exp = ExperimentAutoencoder(DatasetProviderClass(FashionMnistDataset), basic_model_provider, generate_data_for_trainig)
exp.train(BasicTrainParameters(1, 128, True, 'mylog.json', True) )



In [0]:
from experiments import *
exp = ExperimentAutoencoder(DatasetProviderClass(FashionMnistDataset), basic_model_provider, generate_data_for_trainig)
# exp.train(BasicTrainParameters(1, 128, True, 'mylog.json', True) )
# exp.train(BasicTrainParameters(2, 128, True, 'mylog.json', True) )
train_result = exp.train(BasicTrainParameters(3, 128, True, 'mylog.json', True) )


In [0]:
exp_local = train_result[1]
exp_local

In [0]:
import experiments
import datasets
import importlib
importlib.reload(experiments)
for rate in [1]:
    exp = experiments.ExperimentAutoencoder(experiments.DatasetProviderClass(datasets.FashionMnistDataset), experiments.BasicModelProvider(), experiments.generate_data_for_trainig)
    exp.train(experiments.BasicTrainParametersOneModel(100, 128, 10, True, rate, 0.001, 'mylog.json', True))
    #exp.train(experiments.BasicTrainParametersOneModel(100, 128, True, rate, 'mylog.json', True))

In [0]:
import vizualizer
import experiments
import importlib
importlib.reload(vizualizer)
results = experiments.ExperimentBase.load_experiment_results('classifier.json')
vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)